## ニューラルネットワーク
ニューラルネットワークは`torch.nn`パッケージで構築することができます。
`autograd`は前回のチュートリアルで見たと思いますが，`nn`はモデルを定義したり，そのモデルの微分計算をするので，`autograd`に依存するパッケージです。
`nn.Module`には，層とメソッド`forward(input)`が含まれており，`forward(input)`は`output`をかえします。

例えば，手書き文字を分類するニューラルネットワークを見てみましょう：

![http://pytorch.org/tutorials/_images/mnist.png](http://pytorch.org/tutorials/_images/mnist.png)
convnet

これはシンプルな順伝播ネットワークです。つまり，リカレントニューラルネットワークのように，後の層の出力が前の層に戻ることがない，一方向のネットワークです。

ニューラルネットワークの学習は，大体次のような手続きで行います：
- 学習可能なパラメータをもつニューラルネットワークを定義する。
- データを入力し，入力のたびに学習を繰り返す。
- 損失計算を行う。損失は，教師データと出力がどれだけ離れているかを示す。
- 誤差逆伝播法による勾配計算
- ネットワークのパラメータを更新する。更新式は，次式のようになる：
$w = w - \alpha \nabla L$

### ネットワークを定義する
それでは，実際にネットワークを定義してみましょう：

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


`forward`関数を定義するだけで，`backward`関数は自動的に定義できます。この`backward`関数というのは，勾配が計算される関数です。これが`autograd`の便利なところです。
`forward`関数では，Tensorに関する全ての演算を使うことができます。
***
学習可能なパラメータは`net.parameters()`で確認できます。

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


forward関数の入力も出力も両方`autograd.Variable`です。
上記のネットワーク(LeNet)の入力は32×32なので，もしMNISTデータセットでこのネットワークを使うなら32×32にリサイズする必要があります。
***
全てのパラメータの勾配情報を溜め込んでいるバッファーを0にし，ランダムな勾配を逆伝播させるには：

In [3]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0469  0.0944 -0.0326  0.1442 -0.0253  0.0428  0.0319  0.0414 -0.0681  0.0572
[torch.FloatTensor of size 1x10]



Note:
`torch.nn`はミニバッチのみサポートしているので，単体のデータを利用できません。
例えば，`nn.Conv2d`は４次元テンソル`nSamples x nChannels x Height x Width`を引数としてとります。単体のサンプルを入力するときは，`input.unsqueeze(0)`を使うことでバッチの次元をごまかすことができます。

***

一度，これまでのおさらいをします。

- `torch.Tensor` - 多次元配列
- `autograd.Variable` - Tensorのラッパーで，Tensorに行われた演算を記録する。`Tensor`と同様のAPIを持ち，`backward()`のような追加のメソッドなどもある。tensorに関する勾配情報も保持している。
- `nn.Module` - ニューラルネットワークのモジュール。パラメータをカプセル化したり，パラメータをGPUに載せたり，出力したり，ロードしたりする便利な機能をもつ。
- `nn.Parameter` - Variableの一種で，`Module`を定義した時に自動的に登録される。
- `autograd.Function` - autogradの演算における順伝播と逆伝播の定義を実装したパッケージ。すべての`Variable`の演算は，少なくとも１つの`Function`ノードを作る。そしてこのノードは，`Variable`を生成した関数と結び付けられる。そしてこのノードが，演算の履歴を再現する。

ここまで，
- ニューラルネットワークを定義する
- 入力の処理とbackwardの呼び出し
について見てきました。

次に，
- 損失の計算
- ネットワークの更新
について見ていきます。

## 損失関数
損失関数は (output, target) という２つの入力を受け取り，outputがtargetとどれだけ離れているかを計算します。

`nn`パッケージには，いくつかの損失関数が実装されています。シンプルな損失関数として，`nn.MSELoss`を取り上げてみます。これは，入力とターゲットの平均二乗誤差を計算するものです。

例えば，

In [4]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.2517
[torch.FloatTensor of size 1]



以下は，`loss`がどのように計算されているかについてのフローです。

    input  -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d

           -> view -> linear -> relu -> linear -> relu -> linear

           -> MSELoss

           -> loss